In [15]:
import dask.dataframe as dd
from dask_ml.preprocessing import DummyEncoder, Categorizer
from dask_glm.datasets import make_classification
from dask_ml.linear_model import LinearRegression as dask_LinearRegression

from sklearn import datasets, linear_model
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import r2_score, mean_squared_error

Importing the data

In [2]:
raw = dd.read_csv("https://gist.githubusercontent.com/aromerovilla/7170e4ff45dd943af6a920d2f510cd0f/raw/b5732e3122ca9d0dec95717b62434e2b68e642b6/hour.csv")

Glimps of the data

In [3]:
raw.info()
raw.dtypes

<class 'dask.dataframe.core.DataFrame'>
Columns: 17 entries, instant to cnt
dtypes: object(1), float64(4), int64(12)

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [4]:
raw.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Make a pipeline

In [5]:
selected = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', ]

In [6]:
pipeline = make_pipeline(Categorizer(columns=selected), DummyEncoder(columns=selected))
data = pipeline.fit_transform(raw)

Ceate train and test

In [7]:
X = data.loc[:, data.columns != 'cnt']
y = data.loc[:, ['instant','cnt']]

In [8]:
X_train = X.loc[(X.instant <= 15211)].drop(['dteday', 'instant', 'registered','casual'], axis=1)
y_train = y.loc[(y.instant <= 15211)].drop('instant', axis=1)
X_test = X.loc[(X.instant > 15211)].drop(['dteday', 'instant', 'registered','casual'], axis=1)
y_test = y.loc[(y.instant > 15211)].drop('instant', axis=1)

Convert train and test into arrays

In [9]:
X_train_a = X_train.values
X_test_a = X_test.values
y_train_a = y_train.values
y_test_a = y_test.values

Model

In [16]:
l_model = dask_LinearRegression()
l_model.fit(X_train_a, y_train_a)

/Users/albertolombatti/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
/Users/albertolombatti/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multi_class='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)

In [17]:
l_model.score(X_test_a, y_test_a).compute()

/Users/albertolombatti/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices


57614.92966233181

In [20]:
lm_preds = l_model.predict(X_test_a)
print(r2_score(y_test_a.compute(), lm_preds.compute()))

/Users/albertolombatti/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
/Users/albertolombatti/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


0.6366230638977766
